In [3]:
from langchain.tools import Tool
from langchain.utilities.wikipedia import WikipediaAPIWrapper
from langchain.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain.document_loaders.web_base import WebBaseLoader
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

def save_to_txt(text: str, filename: str = "research.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
    return f"Research saved to {filename}"

wiki_tool = Tool(
    name="Wikipedia",
    func=WikipediaAPIWrapper().run,
    description="Searches and returns results from Wikipedia"
)
duck_tool = Tool(
    name="DuckDuckGo",
    func=DuckDuckGoSearchRun().run,
    description="Searches and returns results from DuckDuckGo"
)
def web_scrape(url: str) -> str:
    try:
        # User-Agent 추가
        session = requests.Session()
        session.headers.update({
            "User-Agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"
        })
        loader = WebBaseLoader(url, session=session)
        docs = loader.load()
        return docs[0].page_content[:2000]
    except Exception as e:
        return f"Failed to load content from {url}: {str(e)}"
web_tool = Tool(
    name="WebScraper",
    func=web_scrape,
    description="Extracts and returns main text from a website URL"
)
save_tool = Tool(
    name="SaveToTxt",
    func=save_to_txt,
    description="Saves research content to a .txt file"
)

llm = ChatOpenAI(temperature=0, model="gpt-4")

tools = [wiki_tool, duck_tool, web_tool, save_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

query = "Research about the XZ backdoor"
result = agent.run(query)
print(result)




> Entering new AgentExecutor chain...
I need to find information about the XZ backdoor. I'll start by searching for it on Wikipedia.
Action: Wikipedia
Action Input: XZ backdoor
Observation: Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres F